
# ACSE-7 (Inversion and Optimisation)

# Homework Lecture 8: Data Assimilation (answers) 


### Probabilities


 1. Using the identity $\left(\sum_{i=1}^n a_i\right)^2=\sum_{i=1}^n a_i^2+\sum_{i=1}^n\sum_{j\neq i} a_i  a_j$ we have
$$ E\left(\left[\sum_i X_i^2\right]-\frac{1}{n}\left[\sum_i X_i\right]^2\right) = E\left(\left[\sum_i X_i^2\right]-\frac{1}{n}\left[\sum_i X_i^2\right]-\frac{1}{n}\sum_{i=1}^n\sum_{j\neq i} X_i X_j\right) $$
Since $E$ commutes with addition we may then write
$$ E\left(\left[\sum_i X_i^2\right]-\frac{1}{n}\left[\sum_i X_i\right]^2\right) = \left(1-\frac{1}{n}\right)\left[\sum_i E\left(X_i^2\right)\right]-\frac{1}{n}\sum_{i=1}^n\sum_{j\neq i} E\left(X_i X_j\right).$$
Now using the definitions of $\sigma^2$, and $\mu$ and the fact that $X_i$ and $X_j$ are independent for $i\neq j$,
$$\begin{eqnarray*} E\left(\left[\sum_i X_i^2\right]-\frac{1}{n}\left[\sum_i X_i\right]^2\right) =& \left(1-\frac{1}{n}\right)\sum_i \left[ \sigma^2 +\mu^2 \right]+\frac{1}{n}\sum_{i=1}^n\sum_{j\neq i} E\left(X_i\right) E\left(X_j\right),\\
=& \left(1-\frac{1}{n}\right)\sum_i \left[ \sigma^2 +\mu^2 \right]-\frac{1}{n}\sum_{i=1}^n\sum_{j\neq i} \mu^2.\end{eqnarray*}$$
Finally, doing the summations and cancelling the $\mu$s
$$ E\left(\left[\sum_i X_i^2\right]-\frac{1}{n}\left[\sum_i X_i\right]^2\right) = (n-1)\sigma^2 $$

2. Multiplying out,
$$ E \left( [X-\mu_X][Y-\mu_Y]\right) = E(XY -\mu_XY-X\mu_Y+\mu_X\mu_Y).$$
Since $E$ commutes with addition and with scalar multiplication
$$ E \left( [X-\mu_X][Y-\mu_Y]\right) = E(XY) -\mu_XE(Y)-E(X)\mu_Y+\mu_X\mu_Y.$$
Now $E(X)=\mu_X$ and $E(Y)=\mu_Y$, hence as claimed
$$ E \left( [X-\mu_X][Y-\mu_Y]\right) = E(XY) -\mu_X\mu_Y$$
3. One possible implementation

In [1]:
import numpy as np
import matplotlib.pyplot as p

def polar_normal_algorithm(mu=0, sigma=1.0, N=1):
    counter = 0
    tol = 1.0e-16
    a = np.empty(N)
    b = np.empty(N)
    while counter<N:
        u, v = np.random.uniform(-1, 1, size=2)
        s = u**2+v**2
        if s<tol or s>1.0:
            continue
        c = np.sqrt(-2*np.log(s)/s)
        a[counter] = sigma*u*c+mu
        b[counter] = sigma*v*c+mu
        counter += 1
    return a, b

a, b = polar_normal_algorithm(1, 2, N=10000)

p.hist(a, density=True, alpha=0.4)
p.hist(b, density=True, alpha=0.4)
x = np.linspace(-10, 10)
def p_x(x):
    return 1/np.sqrt(8*np.pi)*np.exp(-(x-1)**2/8.0)
p.plot(x, p_x(x));
        

### The BLUE

1. For the case of $n$ independent unbiased observations, $T_i$ of a variable, $T$, we can write our linear estimator as
$T_{BLUE} = \sum_{i=1}^n a_i T_i.$
Defining our errors as $e_i = T_i -T$, the expected error in $T_{BLUE}$ is
$$E(T_{BLUE}-T) = E\left(T\left(1-\sum_{i=1}^n a_i\right) +\sum_{i=1}^n a_i e_i\right)=T\left(1-\sum_{i=1}^n a_i\right),$$
so to be unbiased $\sum_{i=1}^n a_i=1$. Now for unbiased estiamtesthe expectation of the square error is
$$E((T_{BLUE}-T)^2) = E((\sum_{i=1}^n a_i e_i)^2)=\sum_i a_i^2 \sigma_i^2,$$
where $\sigma_i^2$ is the variance of the error in the $i$th observation and we have used the independence of the errors. We'll minimise this using a Lagrange multiplier, so
$$\mathcal{J}=\lambda\left(1-\sum_{i=1}^n a_i\right)+\sum_i a_i^2 \sigma_i^2$$
and our optimal choice satisfies
$$ \sum_{i=1}^n a_i = 1, $$
$$ 2a_i\sigma_i^2 = \lambda. $$

The second equation implies $a_i=\frac{\lambda}{\sigma_i^2}$ while the first then gives
$$\lambda\left[\sum\frac{1}{\sigma_i^2}\right]=1\Rightarrow \lambda = \frac{1}{\sum_{i=1}^n\frac{1}{\sigma_i^2}}$$ so that
$$ a_i = \frac{1}{\sum_{j=1}^n\frac{\sigma_i^2}{\sigma_j^2}}.$$
Finally, our BLUE is
$$T_{BLUE} = \frac{1}{\sum_{j=1}^n\frac{1}{\sigma_j^2}}\sum_{i=1}^n \left[\frac{T_i}{\sigma_i^2}\right].$$
2. Consider our usual linear combination of our observations,
$$ T_\mbox{guess}=\sum_i a_i T_i = \sum_i \left[a_i \left(T+e_i\right)\right]. $$
So that (for independent observation errors) our mean square error is
$$E(e^2):=E(T_\mbox{guess}-T_t)=\sum_i a^2_i \sigma_i-2 +(1-\sum_i a_i)^2  T^2 $$ 
dropping the constraint that the estimator must be unbiased, and thus the $a_i$s must sum to 1, we obtain optimality constraints,
$$ a_i(T_t^2+\sigma_i^2) = (1-\sum_j a_j)T_t^2. $$

Since all the right hand sides are the same for every $i$, we can just call it $\lambda$ to get $a_i = \frac{\lambda}{T_t^+\sigma_i^2}$ and thus
$$ \lambda = \left(1-\lambda\sum_i \frac{1}{T_t^2+\sigma_i^2}\right) $$
or in other words,
$$ a_i = \frac {1}{T_t^2 + \sigma_i}.$$

Although this choice of $a_i$ does indeed have a smaller error than the BLUE, it also depends explicitly on knowledge of $T_t$, which is precisely the thing we are trying to find. This makes it impossiple to use, unless we happen to know $T_t$. As such, this is not a useful realworld estimtor



### Optimal interpolation

Questions 1 & 2 are purely practical exercises. The real take home here is that data assimilation methods cease to minimise the mean square error in analysis states (or, for variational approaches, to generate a maximum likelihood estimator) as soon as the statitics inserted into them are invalid. In particular, systematic biases need to be identified, or they can significntly skew your analysis estimates.

3. In the real world, "copy-pasted" observations from the same instrument will be perfectly correlated. This means that the $\mathbf{R}$ matrix will have a pair of rows (also columns) which are identical (i.e. not linearly independent). This makes the matrix singular, as you learnt in Lecture 1. Fortunately in this case the "right hand side", that is the relevant columns of $\mathbf{y}-\mathbf{H}x$ will also be identical, since the $\mathbf{H}$ operator should be the same.  This means we are in the situation with infinite solutions, rather than the more inconvenient case with zero solutions, and in fact the $\mathbf{H}^T$ operator will fix most of our problems for us, so we could obtain the result we'd expect using a pseudom inverse.

### 3D-Var

Here is some sample code with 

In [2]:
# Example 3D-Var implementation and solution

# We will use some weather-like 2d data and generate the B from climatology.

import numpy as np
from scipy.sparse.linalg import cg
from scipy.optimize import minimize

nx = 26
ny = 11

Lx = 1e6
Ly = 4e5

U_0 =  30.0
radius = 5e4

def wind_field(X, Y, circulations, centres):
    
    U = np.full((ny, nx), U_0)
    V = np.zeros((ny, nx))
    
    for circ, (x, y) in zip(circulations, centres):
        
        r2= (X-x)**2 + (Y-y)**2
        
        u = circ/(2*np.pi)*np.where(r2>radius**2, 1./r2, 1.0/radius**2) 
        
        
        U -= (Y-y)*u
        V += (X-x)*u
        
    return U, V

X, Y = np.meshgrid(np.linspace(0,Lx,nx), np.linspace(0,Ly, ny))


def random_vortices(N, kx=5, ky=5):
    return (200*np.random.lognormal(0, 0.1, size=N)*radius,
            np.random.uniform([-kx*radius, -kx*radius], [Lx+ky*radius, Ly+ky*radius], (N, 2)))

U_t, V_t = wind_field(X, Y, *random_vortices(4, -1, -1))

# observation locations
n_full = 25
n_speed = 25
y_loc = np.random.randint(0, nx*ny, n_full+n_speed)
# observation values
y = np.empty(2*n_full+n_speed)
y[:n_full] = U_t.ravel()[y_loc[:n_full]] + np.random.normal(0, 2.0, n_full)
y[n_full:2*n_full] = V_t.ravel()[y_loc[:n_full]] + np.random.normal(0, 2.0, n_full)
y[2*n_full:] = (np.sqrt(U_t.ravel()[y_loc[n_full:]]**2
                      + V_t.ravel()[y_loc[n_full:]]**2)
                      + np.random.normal(0, 2, n_speed))

In [11]:
def h(x):
    hx = np.empty(2*n_full+n_speed)
    u = x[y_loc]
    v = x[ny*nx+y_loc]
    hx[:n_full] = u[:n_full]
    hx[n_full:2*n_full] = v[:n_full]
    hx[2*n_full:] = np.sqrt(u[n_full:]**2+v[n_full:]**2)
    
    return hx

def H(x):
    H_x = np.zeros((2*n_full+n_speed, 2*nx*ny))
    u = x[y_loc]
    v = x[ny*nx+y_loc]
    
    # To avoid problems with division by zero, we add a small
    # factor to the denominator.
    vel = np.sqrt(u[n_full:]**2+v[n_full:]**2)+1.0e-16
    for _ in range(n_full):
      H_x[_, y_loc[_]] = 1.0
      H_x[n_full+_, ny*nx+y_loc[_]] = 1.0
    for _ in range(n_speed):
      H_x [2*n_full+_, y_loc[n_full+_]] = u[n_full+_]/vel[_]
      H_x [2*n_full+_, ny*nx+y_loc[n_full+_]] = v[n_full+_]/vel[_]
    
    return H_x
    

R = 2.0**2*np.eye(2*n_full+n_speed)

U = np.empty((5000,ny,nx))
V = np.empty((5000,ny,nx))

for _ in range(U.shape[0]):
    U[_, : :], V[_, :, :] = wind_field(X, Y, *random_vortices(4))

mu_u = np.mean(U, 0)
mu_v = np.mean(V, 0)

d = np.empty((U.shape[0], 2*ny*nx))
for _ in range(d.shape[0]):
    d[_, :ny*nx] = (U[_, :]-mu_u).ravel() 
    d[_, ny*nx:] = (V[_, :]-mu_v).ravel()
    
B = np.empty((2*nx*ny, 2*nx*ny))
for i in range(2*nx*ny):
    for j in range(2*nx*ny):
        B[i, j] = np.sum(d[:, i]*d[:, j])/(U.shape[0]-1)
        
x_b = np.empty(2*ny*nx)

x_b[:ny*nx] = mu_u.ravel()
x_b[ny*nx:] = mu_v.ravel()

In [13]:
print(H(x_b).nonzero())

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       50, 51, 51, 52, 52, 53, 53, 54, 54, 55, 55, 56, 56, 57, 57, 58, 58,
       59, 59, 60, 60, 61, 61, 62, 62, 63, 63, 64, 64, 65, 65, 66, 66, 67,
       67, 68, 68, 69, 69, 70, 70, 71, 71, 72, 72, 73, 73, 74, 74]), array([248, 244,  81, 125, 274,  76,  33, 262,  20, 158, 251, 134,  23,
        44, 254,  41, 125, 172,  49,  64, 150, 259,  45, 153, 154, 534,
       530, 367, 411, 560, 362, 319, 548, 306, 444, 537, 420, 309, 330,
       540, 327, 411, 458, 335, 350, 436, 545, 331, 439, 440, 137, 423,
       236, 522, 254, 540,  28, 314, 156, 442, 254, 540, 203, 489, 150,
       436,  91, 377, 217, 503,  23, 309, 155, 441,  72, 358, 279, 565,
        46, 332, 208, 494, 118, 404, 234, 520,  18, 304, 105, 391, 141,
       427,  16, 302,  82, 368,  31, 317,  63, 349]

In [19]:
def Binv(z0):
    "Iterative solver for inverse of B matrix."
    z = z0/B.diagonal()
    z, info = cg(B, z0, z, maxiter=10, atol=1e-8)
    return z

def Rinv(z0):
    "Fast diagonal solver for R matrix."
    return z0/R.diagonal()


def J(x):
    dx_b = x-x_b
    dx_o = y - h(x)
    return np.dot(dx_b, Binv(dx_b))+np.dot(dx_o, Rinv(dx_o))

def jac(x):
    dx_b = x-x_b
    dx_o = y - h(x)
    
    return 2.0*Binv(dx_b)-2.0*H(x).T.dot(Rinv(dx_o))

def callback(x):
    print(J(x))

In [21]:
# Method using finite difference gradients
%time res = minimize(J, x_b, method='CG', tol=1e-3, options={'maxiter':10})
print('J for x_a:', J(res.x))
# Method provided a Jacobian
%time res = minimize(J, x_b,  jac=jac, method='CG', tol=1e-3, options={'maxiter':100})
x_a = res.x

CPU times: user 1min 25s, sys: 2min 16s, total: 3min 42s
Wall time: 34.6 s
J for x_a: 108.51737703511122
CPU times: user 215 ms, sys: 505 ms, total: 720 ms
Wall time: 107 ms


In [22]:
print('J for x_b:', J(x_b))
print('J for x_a:', J(x_a))

J for x_b: 2054.4771987630143
J for x_a: 98.6461129804279


4. The equations are
$$ \mathbf{R}{a} + H\mathbf{b} = \mathbf{c},$$
$$ \mathbf{H}^T\mathbf{a}- \mathbf{B}^{-1}a\mathbf{b} = \mathbf{0}.$$
Eliminating $\mathbf{b}$ gives $\mathbf{a}= (\mathbf{R}+\mathbf{H}\mathbf{B}\mathbf{H}^T)^{-1}\mathbf{c}$. Substituting into the second equation gives
$$\mathbf{b} = \mathbf{B}\mathbf{H}^T(\mathbf{R}+\mathbf{H}\mathbf{B}\mathbf{H}^T)^{-1}\mathbf{c}.$$
Eliminating $\mathbf{a}$ directly gives 
$$\mathbf{b} = (\mathbf{B}^{-1}+\mathbf{H}^T\mathbf{R}^{-1}\mathbf{H})^{-1}\mathbf{H}^T\mathbf{R}^{-1}\mathbf{c}.$$
These are the Optimal interpolation and 3D-Var forms of the increment to the background state. Since the problem is well posed, the two methods must generate the same solution, proving that linear 3D-Var and Optimal Interpolation find the same solution.

### Kalman Filter

1. The basic Kalman update equations for hindcasting become 
  $$\mathbf{x}_a^{(k)}= \mathbf{x}_b^{(k)}+\mathbf{B}^{(k)}\mathbf{H}^T(\mathbf{R}^{(k)}+\mathbf{H}\mathbf{B}^{(k)}\mathbf{H}^T)^{-1}(\mathbf{y}-\mathbf{H}\mathbf{x}_b^{(k)}) $$
  $$\mathbf{P}^{(k)} = (\mathbf{I}-\mathbf{B}^{(k)}\mathbf{H}^T(\mathbf{R}^{(k)}+\mathbf{H}\mathbf{B}^{(k)}\mathbf{H}^T)^{-1}\mathbf{H})\mathbf{B}^{(k)}.$$
  $$\mathbf{x}_b^{(k-1)} = \mathbf{M}^{-1} \mathbf{x}_a^{(k)}.$$
  $$\mathbf{B}^{(k-1)} = \mathbf{M}^{-1} \mathbf{P}^{(k)} \left[\mathbf{M}^{-1}\right]^T+\mathbf{Q}.$$
  


### 4D-Var

 1. Rewriting $\mathbf{x}_0$ as $\mathbf{M}^{-1}\mathbf{x}_1$ gives a cost function wholy in $\mathbf{x}_1$,  
$$\mathcal{J}(\mathbf{x}_1) = \frac{1}{2}(\mathbf{M}^{-1}\mathbf{x}_1-\mathbf{x}_b)^T\mathbf{B}^{-1}(\mathbf{M}^{-1}\mathbf{x}_1-\mathbf{x}_b) + \frac{1}{2}(\mathbf{y}_1-\mathbf{H}\mathbf{x}_1)^T\mathbf{R}^{-1}(\mathbf{y}_1-\mathbf{H}\mathbf{x}_1).$$
Differentiating this function with respect to $\mathbf{x}_1$ gives
$$\nabla_{\mathbf{x}_1}\mathcal{J}=[\mathbf{M}^{-1}]^T\mathbf{B}^{-1}(\mathbf{M}^{-1}\mathbf{x}_1-\mathbf{x}_b)+\mathbf{H}^T\mathbf{R}^{-1}(\mathbf{y}_1-\mathbf{H}\mathbf{x}_1).$$
The optimal solution is therefore
$$ \mathbf{x}_a^(1) = \mathbf{M} \mathbf{x}_b + \mathbf{M}\mathbf{B}\mathbf{M}^T\mathbf{H}^T(\mathbf{R}+\mathbf{H}\mathbf{M}\mathbf{B}\mathbf{M}^T\mathbf{H}^T)^{-1}(\mathbf{y}-\mathbf{H}\mathbf{M}\mathbf{x}_b),$$
implying that the optimal starting value is
$$\mathbf{x}_a^{(0)} = \mathbf{M}^{-1}\mathbf{x}_a^{(1)}=\mathbf{x}_b+\mathbf{B}\mathbf{M}^T\mathbf{H}^T(\mathbf{R}^{(k)}+\mathbf{H}\mathbf{M}\mathbf{B}\mathbf{M}^T\mathbf{H}^T)^{-1}(\mathbf{y}-\mathbf{H}\mathbf{M}\mathbf{x}_b)$$
This implies that the 4D-Var equation can be interpretted in a number of ways:
  - The first equation is a 3D-Var equation for an error covariance matrix $\mathbf{M}\mathbf{B}\mathbf{M}^T$ (remember that for matrices $[ABC^T]^{-1} = [C^T]^{-1}B^{-1}A^{-1}$)
and a background state $\mathbf{M}\mathbf{x}_b$. In this viewpoint, the forward model is being used pull forward the background state forward and update the error covariance in line with the Kalman gain equations, while the inverse model is used to pull final the analysis state back to the start.
  - Alternatively, the system can be seen as pulling the observation operator forward in time (note the repeated  $\mathbf{H}\mathbf{M}$), with the background error covariance unchanged.
 
 This duality is the basis of the [Kalman Smoother](https://en.wikipedia.org/wiki/Kalman_filter#Fixed-interval_smoothers), which takes the Kalman filter approach and extends it over a fixed number number of data assimilation steps to provide the increment to the *initial* state to minimize the mean square error in the *final* analysis error covariance, rather than working state to state. Linear 4D-Var is thus equivalent to the Kalman smoother in the same way that linear 3D-Var is equivalent to Optimal interpolation.